## Analyse des données public sur l'épidémie de covid19 en France

Source: [https://www.data.gouv.fr/fr/datasets/r/929dff1b-b07c-4637-9690-fb7219ad3eb8]

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('./data/donnees-hospitalieres-classe-age-covid19-2020-09-26-19h00.csv', sep=';',parse_dates=['jour'])

In [3]:
df

,reg,cl_age90,jour,hosp,rea,rad,dc
0,1,0,2020-03-18,0,0,0,0
1,1,9,2020-03-18,0,0,0,0
2,1,19,2020-03-18,0,0,0,0
3,1,29,2020-03-18,0,0,0,0
4,1,39,2020-03-18,0,0,0,0
...,...,...,...,...,...,...,...
38011,94,59,2020-09-26,4,0,41,1
38012,94,69,2020-09-26,4,1,44,4
38013,94,79,2020-09-26,10,2,66,16
38014,94,89,2020-09-26,1,0,45,30


## Analyse temporelle par classe d'âge

Analyse de la dynamique des décès par classe d'àge.

In [4]:
import numpy as np
import pandas_bokeh as pb

In [5]:
pb.output_notebook()
pd.set_option('plotting.backend', 'pandas_bokeh')
# Create Bokeh-Table with DataFrame:
from bokeh.models.widgets import DataTable, TableColumn
from bokeh.models import ColumnDataSource

Loading BokehJS ...

In [6]:
deces_par_age = df[df['cl_age90']>0].pivot_table(index='jour', columns='cl_age90', values='dc',aggfunc='sum')
deces_par_age.plot_bokeh(kind="line",title ="Covid - répartition des décès cumulés",
                   figsize =(800,800),
                        xlabel='Date',
                        ylabel='Décès')

Figure(id='1002', ...)

## Analyse de la situation actuelle

Analyse simplifiée retenant deux groupes d'àge:
* moins de 60 ans
* plus de 60 ans

In [7]:
deces_derniere_situation = df[df['cl_age90']>0].drop(['hosp','rea','rad'], axis=1).set_index(['jour']).last('1D').drop('reg',axis=1).groupby(['jour','cl_age90']).agg('sum').reset_index()

In [8]:

deces_moins_60 = deces_derniere_situation[deces_derniere_situation['cl_age90'] < 69]['dc'].sum()
deces_plus_60 = deces_derniere_situation[deces_derniere_situation['cl_age90'] >= 69]['dc'].sum()

In [12]:
pd.DataFrame(np.array([[deces_moins_60], [deces_plus_60]]),
                   columns=['deces'],index=['moins de 60','plus de 60']).plot_bokeh(kind="pie",title ="Covid - répartition des décès cumulés",
                   figsize =(600,600),xlabel='groupe d\'àge')

__x__values_original


Figure(id='1850', ...)

## Analyse comparée avec la démographie française

[https://www.insee.fr/fr/statistiques/4515539?sommaire=4516122]


In [13]:
demo = pd.read_csv('data/BTT_TD_POP1A_2017.CSV',sep=';')

/usr/local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
demo_agg = demo.drop(['NIVGEO','CODGEO','LIBGEO','SEXE'],axis=1).groupby('AGEPYR10').agg('sum')
demo_agg

,NB
AGEPYR10,
0,2.343591e+06
3,2.505525e+06
6,4.322077e+06
11,6.021670e+06
18,5.751486e+06
25,1.293626e+07
40,1.395209e+07
55,8.734819e+06
65,9.320988e+06


### Basé sur le dernier recensement

* calcul de la population totale
* calcul de la population des moins de 55 ans
* calcul de la population des plus de 55 ans



In [15]:
nb_total = demo['NB'].sum()
nb_moins_55 = demo_agg.loc[0:55].sum()['NB']
nb_plus_55 = demo_agg.loc[65:].sum()['NB']

In [16]:
print("population total: {:.0f}".format(nb_total))
print("population moins de 55 ans {:.0f}".format(nb_moins_55))
print("population plus de 55 ans {:.0f}".format(nb_plus_55))

population total: 70091267
population moins de 55 ans 56567511
population plus de 55 ans 13523756


In [17]:
pd.DataFrame(np.array([[deces_moins_60], [deces_plus_60]]),
                   columns=['deces'],index=['moins de 60','plus de 60'])

,deces
moins de 60,1378
plus de 60,19512


In [18]:
res_agg = pd.DataFrame(np.array([[nb_moins_55,deces_moins_60], [nb_plus_55,deces_plus_60]]),
                   columns=['population','deces'],index=['moins de 55/60','plus de 55/60']).astype(int)

In [19]:
stacked_bar = res_agg.plot_bokeh.bar(
    ylabel="population", 
    title="Impact du covid19 par catégorie d'àge", 
    alpha=0.6,
    xlabel='categorie')